# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math
import re
import os
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas
import requests
import sqlalchemy as db

In [2]:
# any general notebook setup, like log formatting

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [X] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [X] Taxi data:
    * [X] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [X] Load the data using `pandas`
    * [X] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [X] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [4]:
def calculate_distance(from_coord, to_coord): # from_coord, to_coord = [lat, lon]
    # radius of earth in km
    R = 6371

    lat_from = math.radians(from_coord[0])
    lon_from = math.radians(from_coord[1])
    lat_to = math.radians(to_coord[0])
    lon_to = math.radians(to_coord[1])

    # Haversine formula: https://en.wikipedia.org/wiki/Haversine_formula
    dlon = lon_to - lon_from
    dlat = lat_to - lat_from
    a = math.sin(dlat / 2)**2 + math.cos(lat_from) * math.cos(lat_to) * math.sin(dlon / 2)**2
    d = 2 * R * math.asin(math.sqrt(a))
    
    return d   
    

In [5]:
def add_distance_column(dataframe): 
    # dataframe contains pickup_latitude etc. after column name normalization   
    dataframe['distance'] = dataframe.apply(
        lambda x: calculate_distance([x["pickup_latitude"],x["pickup_longitude"]],
                                     [x["dropoff_latitude"],x["dropoff_longitude"]]),
        axis = 1
    )

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [6]:
# should be move to top later 
import warnings
warnings.filterwarnings("ignore")

In [7]:
# can be deleted
zones = geopandas.read_file("taxi_zones.shp")
zones = zones.to_crs(4326)
zones['lon'] = zones.centroid.x
zones['lat'] = zones.centroid.y
zones = zones[["zone", "LocationID", "lon", "lat"]]
zones.head()

,zone,LocationID,lon,lat
0,Newark Airport,1,-74.174000,40.691831
1,Jamaica Bay,2,-73.831299,40.616745
2,Allerton/Pelham Gardens,3,-73.847422,40.864474
3,Alphabet City,4,-73.976968,40.723752
4,Arden Heights,5,-74.188484,40.552659


In [8]:
def get_taxi_html():
    response = requests.get(TAXI_URL)
    html = response.content
    return html

In [9]:
def find_taxi_csv_urls():
    pattern = r"Yellow Taxi Trip Records"
    content = get_taxi_html()
    soup = bs4.BeautifulSoup(content, 'html.parser')
    y_urls = [y['href'] for y in soup.find_all(title=pattern)]
    # getting links from 2009-01 to 2015-06
    y_urls = y_urls[-84:]
    urls = [item for idx, item in enumerate(y_urls) if idx not in range(6, 12)]
    return urls

In [10]:
def get_and_clean_month_taxi_data(url):
    
    # download taxi data with url and save as dataframe
    df = pd.read_parquet(url, engine='pyarrow')
    
    # looking up the latitude and longitude for some months where only location IDs and normalize column names
    pattern1 = r"LocationID"
    pattern2 = r"latitude"
    pattern3 = r"Lat"
    df_type = 0
    for col in df.columns:
        if re.search(pattern1, col):
            df_type = 1 # 2011-01 to 2015-06          
        if re.search(pattern2, col):
            df_type = 2 # 2010-01 to 2010-12
        if re.search(pattern3, col):
            df_type = 3 # 2009-01 to 2009-12
            
    if df_type == 1:
        # load and convert zone data
        zones = geopandas.read_file("taxi_zones.shp")
        zones = zones.to_crs(4326)
        zones['lon'] = zones.centroid.x
        zones['lat'] = zones.centroid.y
        zones = zones[["LocationID", "lon", "lat"]]

        # drop unnecessary columns
        cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime','passenger_count', 'trip_distance', 'PULocationID', 
        'DOLocationID', 'fare_amount','tip_amount']
        df = df[cols]
        
        # get pickup lattitude and longitude
        df = df.merge(zones, left_on='PULocationID', right_on='LocationID', how='left')
        df.drop(["LocationID"], axis=1, inplace=True)
        df.rename({"lon": "pickup_longitude", "lat": "pickup_latitude"}, axis=1, inplace=True)
        
        # get dropoff lattitude and longitude
        df = df.merge(zones, left_on='DOLocationID', right_on='LocationID', how='left')
        df.drop(["LocationID"], axis=1, inplace=True)
        df.rename({"lon": "dropoff_longitude", "lat": "dropoff_latitude"}, axis=1, inplace=True)
        
        # remove unnecessary columns
        df.drop(["PULocationID", "DOLocationID"], axis=1, inplace=True)
        
        # remove nan values
        df.dropna(inplace=True)

        # normalize column names and types
        df.rename(columns={'tpep_pickup_datetime': 'pickup_datetime', 
                           'tpep_dropoff_datetime': 'dropoff_datetime'},
                  inplace=True)
           
    if df_type == 2:
        cols = ['pickup_datetime', 'dropoff_datetime', 'passenger_count','trip_distance', 'pickup_longitude', 
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','fare_amount', 'tip_amount']
        df = df[cols]
        
    if df_type == 3:
        cols = ['Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime','Passenger_Count', 'Trip_Distance', 'Start_Lon', 
                'Start_Lat','End_Lon', 'End_Lat','Fare_Amt', 'Tip_Amt']
        df = df[cols]
        
        # normalize column names and types
        df.rename(columns={'Trip_Pickup_DateTime': 'pickup_datetime', 
                           'Trip_Dropoff_DateTime': 'dropoff_datetime',
                           'Passenger_Count': 'passenger_count',
                           'Trip_Distance': 'trip_distance',
                           'Start_Lon': 'pickup_longitude',
                           'Start_Lat': 'pickup_latitude',
                           'End_Lon': 'dropoff_longitude',
                           'End_Lat': 'dropoff_latitude',
                           'Fare_Amt': 'fare_amount',
                           'Tip_Amt': 'tip_amount'
                          },
                  inplace=True)
        
    # drop nan values
    df.dropna(inplace=True)
    
    # remove trips out side the designated coordbox (40.560445, -74.242330) and (40.908524, -73.717047)
    df = df[(df["pickup_latitude"] >= 40.560445) & (df["pickup_latitude"] <= 40.908524)]
    df = df[(df["pickup_longitude"] >= -74.242330) & (df["pickup_longitude"] <= -73.717047)]
    df = df[(df["dropoff_latitude"] >= 40.560445) & (df["dropoff_latitude"] <= 40.908524)]
    df = df[(df["dropoff_longitude"] >= -74.242330) & (df["dropoff_longitude"] <= -73.717047)]
    
    # drop 0 passenger trips
    df = df[df["passenger_count"] != 0]
    
    # drop trips with no fare 
    df = df[df["fare_amount"] != 0]
    
    # drop trips with no distance between dropoff and pickup
    df = df[(df["pickup_latitude"] != df["dropoff_latitude"]) | (df["pickup_longitude"] != df["dropoff_longitude"])]
    df = df[df["trip_distance"] != 0]

    # normalize column types
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"]) 
    df["dropoff_datetime"] = pd.to_datetime(df["dropoff_datetime"]) 
    
    # sampling 2472 data for each month
    df = df.sample(n=2472)
    
    # drop unnecessary columns
    df.drop(['trip_distance'], axis=1, inplace=True)
    df.reset_index(drop=True)
    
    # save to a csv file 
    df.to_csv(url, encoding='utf-8', index=False)
    return df

In [11]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

In [ ]:
#taxi_data = get_and_clean_taxi_data()

In [12]:
taxi_data = pd.read_csv("taxi_data.csv")

In [13]:
taxi_data.drop("Unnamed: 0", axis=1, inplace=True)

In [14]:
taxi_data.head()

,pickup_datetime,dropoff_datetime,passenger_count,fare_amount,tip_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2009-01-01 02:07:42,2009-01-01 02:24:22,2,13.4,2.01,-73.992843,40.737283,-74.012868,40.708458,3.622301
1,2009-01-01 02:42:32,2009-01-01 02:50:20,1,7.4,0.00,-73.972229,40.689693,-73.992919,40.679139,2.102573
2,2009-01-01 03:08:09,2009-01-01 03:41:29,4,35.0,0.00,-74.005598,40.748357,-73.950937,40.596385,17.516012
3,2009-01-01 03:27:33,2009-01-01 03:36:41,1,5.4,0.00,-73.948160,40.778874,-73.960724,40.780947,1.082704
4,2009-01-01 03:43:08,2009-01-01 03:55:29,1,11.4,0.00,-73.960089,40.800659,-73.985232,40.762745,4.717511


In [39]:
taxi_data.shape

(192816, 10)

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [98]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)

    # drop OTC in datetime
    new = df["pickup_datetime"].str.split(" ", n = 2, expand = True)
    df["pickup_datetime"] = new[0] + " " + new[1]
    
    # convert to datetime
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])  
     
    # remove trips out side the designated coordbox (40.560445, -74.242330) and (40.908524, -73.717047)
    df = df[(df["pickup_latitude"] >= 40.560445) & (df["pickup_latitude"] <= 40.908524)]
    df = df[(df["pickup_longitude"] >= -74.242330) & (df["pickup_longitude"] <= -73.717047)]
    df = df[(df["dropoff_latitude"] >= 40.560445) & (df["dropoff_latitude"] <= 40.908524)]
    df = df[(df["dropoff_longitude"] >= -74.242330) & (df["dropoff_longitude"] <= -73.717047)]
    
    # drop 0 passenger trips
    df = df[df["passenger_count"] != 0]
    
    # drop trips with no fare 
    df = df[df["fare_amount"] != 0]
    
    # drop trips with no distance between dropoff and pickup
    df = df[(df["pickup_latitude"] != df["dropoff_latitude"]) | (df["pickup_longitude"] != df["dropoff_longitude"])]
   
    # drop unnecessary columns
    df.drop(['Unnamed: 0', 'key'], axis=1, inplace=True)
    
    # change column order in accordance with taxi data
    column_order = ['pickup_datetime', 'passenger_count', 'fare_amount', 'pickup_longitude',
                    'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']          
    df = df[column_order]
 
    return df

In [16]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    
    return uber_dataframe

In [17]:
uber_dataframe = load_and_clean_uber_data(UBER_CSV)
uber_data = get_uber_data()
avg_mon_sample = int(len(uber_data) / (12 * 6 + 6))
uber_data.head()

,pickup_datetime,passenger_count,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-05-07 19:52:06,1,7.5,-73.999817,40.738354,-73.999512,40.723217,1.683323
1,2009-07-17 20:04:56,1,7.7,-73.994355,40.728225,-73.994710,40.750325,2.457590
2,2009-08-24 21:45:00,1,12.9,-74.005043,40.740770,-73.962565,40.772647,5.036377
3,2009-06-26 08:22:21,3,5.3,-73.976124,40.790844,-73.965316,40.803349,1.661683
4,2014-08-28 17:47:00,5,16.0,-73.925023,40.744085,-73.973082,40.761247,4.475450


In [18]:
# check uber data shape, can be deleted
uber_data.shape

(192828, 8)

In [19]:
# check nan values, can be deleted
uber_data.isna().sum()

pickup_datetime      0
passenger_count      0
fare_amount          0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
distance             0
dtype: int64

In [20]:
# can be deleted 
int(len(uber_data) / (12 * 6 + 6))
            

2472

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [11]:
def clean_month_weather_data_hourly(csv_file):
    const_col = ["DATE","LATITUDE","LONGITUDE"]
    hourly_cols = ["HourlyPrecipitation", "HourlyWindSpeed"]
    daily_col = ["DailyPrecipitation", "DailyAverageWindSpeed", "DailySustainedWindSpeed"]
    # drop nan values
    pass
    # calculate missing hourly data with daily data

In [ ]:
def clean_month_weather_data_daily(csv_file):
    #raise NotImplemented()
    pass

In [ ]:
# extra credit part
'''
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["TODO"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data
    
'''

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
#taxi_data = get_and_clean_taxi_data()
#uber_data = get_uber_data()
#hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [30]:
engine = db.create_engine(DATABASE_URL)

In [31]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
'''HOURLY_WEATHER_SCHEMA = """
TODO
"""
'''

'''
DAILY_WEATHER_SCHEMA = """
TODO
"""
'''

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    passenger_count INTEGER,
    fare_amount FLOAT,
    tip_amount FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    passenger_count INTEGER,
    fare_amount FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
)
"""

In [32]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    #f.write(HOURLY_WEATHER_SCHEMA)
    #f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [33]:
# create the tables with the schema files
with engine.connect() as connection:
    connection.execute(TAXI_TRIPS_SCHEMA)
    connection.execute(UBER_TRIPS_SCHEMA)
    

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [34]:
def write_dataframes_to_table(table_to_df_dict):
    for key, value in table_to_df_dict.items():
        value.to_sql(key, con=engine, if_exists='append', index=False)
    

In [51]:
def write_dataframes_to_table(table_to_df_dict):
    for key in table_to_df_dict.keys():
        table_to_df_dict[key].to_sql(key, con=engine, if_exists='append', index=False)

In [35]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    #"hourly_weather": hourly_data,
    #"daily_weather": daily_data,
}


In [36]:
write_dataframes_to_table(map_table_name_to_dataframe)

In [37]:
# can be deleted
with engine.connect() as connection:
    for row in connection.execute("SELECT * FROM taxi_trips ORDER BY id ASC limit 10"):
        print(row)

(1, '2009-01-01 02:07:42', '2009-01-01 02:24:22', 2, 13.4, 2.01, -73.99284299999998, 40.737283, -74.01286799999998, 40.708458, 3.62230086821533)
(2, '2009-01-01 02:42:32', '2009-01-01 02:50:20', 1, 7.4, 0.0, -73.972229, 40.689693, -73.992919, 40.679139, 2.102573266751841)
(3, '2009-01-01 03:08:09', '2009-01-01 03:41:29', 4, 35.0, 0.0, -74.005598, 40.748357, -73.95093699999998, 40.596385, 17.5160124129064)
(4, '2009-01-01 03:27:33', '2009-01-01 03:36:41', 1, 5.4, 0.0, -73.94816, 40.778874, -73.960724, 40.780947, 1.0827042097946826)
(5, '2009-01-01 03:43:08', '2009-01-01 03:55:29', 1, 11.4, 0.0, -73.96008899999998, 40.800659, -73.98523199999998, 40.762745, 4.717510748237535)
(6, '2009-01-01 04:26:54', '2009-01-01 04:31:52', 1, 5.4, 0.0, -74.00213999999998, 40.732779, -74.006527, 40.738482, 0.7340062925608284)
(7, '2009-01-01 04:32:47', '2009-01-01 04:38:53', 1, 7.4, 0.0, -73.802509, 40.761286, -73.830185, 40.759464, 2.3397777711756422)
(8, '2009-01-01 04:53:24', '2009-01-01 05:12:17', 2,

In [38]:
count = 0
for i in engine.execute("SELECT * FROM taxi_trips"):
    count+=1
    
print(count)

192816


## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [X] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [X] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [X] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [X] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [40]:
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)


### Query 1

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [41]:
TOP_HOUR_TAXI = """
WITH cte_hourly_taxi_trips AS(
    SELECT strftime('%H',  pickup_datetime) AS hour, COUNT(id) AS trip_counts
    FROM taxi_trips
    GROUP BY hour
)
SELECT * FROM cte_hourly_taxi_trips ORDER BY trip_counts DESC 
"""

In [42]:
engine.execute(TOP_HOUR_TAXI).fetchall()

[('19', 12117),
 ('18', 11424),
 ('20', 11389),
 ('21', 11101),
 ('22', 10838),
 ('14', 9721),
 ('17', 9573),
 ('23', 9542),
 ('12', 9436),
 ('13', 9401),
 ('15', 9262),
 ('11', 9076),
 ('09', 8959),
 ('08', 8656),
 ('10', 8635),
 ('16', 7978),
 ('00', 7758),
 ('07', 7066),
 ('01', 5786),
 ('02', 4118),
 ('06', 3917),
 ('03', 3125),
 ('04', 2149),
 ('05', 1789)]

In [43]:
write_query_to_file(TOP_HOUR_TAXI, "top_hour_taxi.sql")

### Query 2

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [44]:
TOP_DAY_UBER = """
WITH cte_daily_uber_trips AS(
    SELECT strftime('%w',  pickup_datetime) AS hour, COUNT(id) AS trip_counts
    FROM uber_trips
    GROUP BY hour
)
SELECT * FROM cte_daily_uber_trips ORDER BY trip_counts DESC 
"""

In [45]:
engine.execute(TOP_DAY_UBER).fetchall()

[('5', 29749),
 ('6', 29213),
 ('4', 28960),
 ('3', 27971),
 ('2', 27152),
 ('0', 25503),
 ('1', 24280)]

In [46]:
write_query_to_file(TOP_DAY_UBER, "top_day_uber.sql")

### Query 3

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [90]:
DIST_95_PERCENTILE = """
WITH cte_distance AS(
    SELECT strftime('%Y-%m-%d', pickup_datetime) as date, distance
    FROM taxi_trips
    WHERE date < '2013-08-01' AND date >= '2013-07-01'
    UNION ALL
    SELECT strftime('%Y-%m-%d', pickup_datetime) as date, distance
    FROM uber_trips
    WHERE date < '2013-08-01' AND date >= '2013-07-01'
),
dist_percentile AS (
    SELECT distance, NTILE(100) OVER(ORDER BY distance ASC) AS percentile
    FROM cte_distance
)
SELECT MAX(distance) FROM dist_percentile WHERE percentile = 95
"""

In [91]:
engine.execute(DIST_95_PERCENTILE).fetchall()

[(10.371616541970583,)]

In [89]:
write_query_to_file(DIST_95_PERCENTILE, "dist_95_percentile.sql")

In [94]:
# test, can be deleted later
DIST_95_PERCENTILE = """
WITH cte_distance AS(
    SELECT strftime('%Y-%m-%d', pickup_datetime) as date, distance
    FROM taxi_trips
    WHERE date < '2013-08-01' AND date >= '2013-07-01'
    UNION ALL
    SELECT strftime('%Y-%m-%d', pickup_datetime) as date, distance
    FROM uber_trips
    WHERE date < '2013-08-01' AND date >= '2013-07-01'
)
SELECT distance FROM cte_distance
"""

In [97]:
# test using df
import numpy as np
l = []
for i in engine.execute(DIST_95_PERCENTILE).fetchall():
    dist, = i
    l.append(dist)
l = np.array(l)
p95 = np.percentile(l, 95)

p95 

10.371616541970583

### Query 4

_**TODO:** Write some prose that tells the reader what you're about to do here._

What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?

In [116]:
TOP_10_RIDE_DAY = """
WITH cte_rides_days AS(
    SELECT strftime('%Y-%m-%d', pickup_datetime) as date, COUNT(id) AS count, SUM(distance) AS dist_sum
    FROM taxi_trips
    WHERE date < '2010-01-01' AND date >= '2009-01-01'
    GROUP BY date
    UNION ALL
    SELECT strftime('%Y-%m-%d', pickup_datetime) as date, COUNT(id) AS count, SUM(distance) AS dist_sum
    FROM uber_trips
    WHERE date < '2010-01-01' AND date >= '2009-01-01'
    GROUP BY date
)
SELECT date, count, dist_sum / count FROM cte_rides_days ORDER BY count DESC LIMIT 10
"""

In [117]:
engine.execute(TOP_10_RIDE_DAY ).fetchall()

[('2009-12-11', 127, 2.975892034720371),
 ('2009-10-23', 121, 2.4930037783117567),
 ('2009-02-06', 119, 3.1583094263812006),
 ('2009-08-14', 118, 3.5786906513189383),
 ('2009-02-28', 117, 2.8639458432343146),
 ('2009-02-13', 113, 3.228174601506392),
 ('2009-04-18', 113, 3.3297941252720924),
 ('2009-07-17', 109, 3.3269789532684335),
 ('2009-11-12', 109, 2.835980708871698),
 ('2009-11-20', 109, 3.19047368312765)]

In [118]:
write_query_to_file(TOP_10_RIDE_DAY, "top_10_ride_day.sql")

In [121]:
# test can be deleted
TOP_10_RIDE_DAY = """
WITH cte_rides_days AS(
    SELECT strftime('%Y-%m-%d', pickup_datetime) as date, id
    FROM taxi_trips
    WHERE date < '2010-01-01' AND date >= '2009-01-01'
    UNION ALL
    SELECT strftime('%Y-%m-%d', pickup_datetime) as date, id
    FROM uber_trips
    WHERE date < '2010-01-01' AND date >= '2009-01-01'
)
SELECT * FROM cte_rides_days
"""

In [123]:
# test can be deleted
count = 0
for i in engine.execute(TOP_10_RIDE_DAY).fetchall():
    count+=1
count

59389

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [19]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    #raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)